In [2]:
import numpy as np 
import pandas as pd 
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from subprocess import check_output

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from score import calc_auc_score, calc_log_loss

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Take Input

In [3]:
# read files
def readInputFiles(train_file_path, test_file_path):
    train = pd.read_csv(train_file_path)
    test = pd.read_csv(test_file_path)
    train = train.sample(frac=1)
    return train, test
    
train, test = readInputFiles('../dataset/train_new.csv', '../dataset/test_new.csv')

## Preprocessing Data

In [4]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

def preProcessData():
    max_features = 20000
    maxlen = 200

    # grab all the comments from train and fill the NAN comments with CVxTz
    list_sentences_train = train["comment_text"].fillna("CVxTz").values
    # grab all the comments from test and fill the NAN comments with CVxTz
    list_sentences_test = test["comment_text"].fillna("CVxTz").values

    # get the values for 6 classes
    y = train[list_classes].values

    tokenizer = text.Tokenizer(num_words=max_features)

    # only use the training data comments for tokenizer
    tokenizer.fit_on_texts(list(list_sentences_train))

    # convert form strings to list of indices of words
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

    # truncate list if length over 100
    # pad list if length less than 100
    X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)
    
    return max_features, maxlen, X_train, X_test, y
    
max_features, maxlen, X_train, X_test, y = preProcessData()

In [5]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [6]:
def createModel(file_path):
    model = get_model()
    batch_size = 32
    epochs = 2

    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # The point is to let the script run for more epochs locally 
    # because it will timeout if done so in Kaggle Kernels
    early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

    callbacks_list = [checkpoint, early] #early
    model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

    model.load_weights(file_path)

    return model

## Fit model if doesn't exist already 

In [7]:
print("start fitting...")
file_path = "bidirectional_lstm_2.h5"
if os.path.isfile(file_path):
    print ("Model already exists. Loading from path ", file_path)
    model = load_model(file_path)
else:
    print ("Model doesn't exist already, training model and saving at path ", file_path)
    model = createModel("weights_base.best.hdf5")
    model.save(file_path)

start fitting...
("Model doesn't exist already, training model and saving at path ", 'bidirectional_lstm_2.h5')
Train on 114890 samples, validate on 12766 samples
Epoch 1/2
114890/114890 [==============================] - 39188s 341ms/step - loss: 0.0686 - acc: 0.9776 - val_loss: 0.0461 - val_acc: 0.9829

Epoch 00001: val_loss improved from inf to 0.04612, saving model to weights_base.best.hdf5
Epoch 2/2
114890/114890 [==============================] - 1273s 11ms/step - loss: 0.0457 - acc: 0.9830 - val_loss: 0.0448 - val_acc: 0.9832

Epoch 00002: val_loss improved from 0.04612 to 0.04480, saving model to weights_base.best.hdf5


In [9]:
print("start prediccting...")
y_pred = model.predict(X_test, batch_size=1024)
print ("done")

start prediccting...
done


In [23]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# create the submission file and store the results
def saveResults():
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for idx, col in enumerate(list_classes):
        submission[col] = y_pred[:,idx]
    submission.to_csv('submission.csv', index=False)
    
saveResults()

In [19]:
def get_scores(test, preds, fallback_preds_filename):
    try: 
        true = test
    except NameError:
        true = pd.read_csv('../dataset/test_new.csv')
    try: 
        y_pred = preds
    except NameError:
        pred = pd.read_csv(fallback_preds_filename)
        y_pred = pred[list_classes].values

    y_true = true[list_classes].values

    loss = calc_log_loss(y_true, y_pred)
    auc = calc_auc_score(y_true, y_pred)
    return loss, auc

In [21]:
true = pd.read_csv('../dataset/test_new.csv')
pred = y_pred

loss_, aucs = get_scores(true, y_pred, fallback_preds_filename=None)

In [22]:
print ("Log loss = ", loss_)
print ("AUC Score = ", aucs)

('Log loss = ', 0.046999667351311114)
('AUC Score = ', 0.9789796280620561)
